In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import os
import torch
import sys

# Add the new Python dispersion library path
dispersion_library_path = r'D:\Research\NO-2D-Metamaterials\2d-dispersion-py'
sys.path.append(dispersion_library_path)

# Import functions from the new Python library
try:
    from wavevectors import get_IBZ_wavevectors, get_IBZ_contour_wavevectors
    from plotting import plot_dispersion, plot_design, plot_dispersion_surface
    from get_design import get_design
    from dispersion import dispersion
    from utils import validate_constants
    print("Successfully imported functions from the new Python dispersion library")
except ImportError as e:
    print(f"Error importing from dispersion library: {e}")
    print("Make sure the 2d-dispersion-py directory is in the correct location")

# Load the new PyTorch dataset structure
dataset_path = r"D:\Research\NO-2D-Metamaterials\data\set_c1_1200n_reduced_wv5_b2"  # Update this path
fn = os.path.basename(dataset_path)

# Load PyTorch tensors
geometries = torch.load(os.path.join(dataset_path, 'geometries_full.pt'), weights_only=False)
waveforms = torch.load(os.path.join(dataset_path, 'waveforms_full.pt'), weights_only=False)
band_ffts = torch.load(os.path.join(dataset_path, 'band_fft_full.pt'), weights_only=False)
displacements_dataset = torch.load(os.path.join(dataset_path, 'displacements_dataset.pt'), weights_only=False)
reduced_indices = torch.load(os.path.join(dataset_path, 'reduced_indices.pt'), weights_only=False)

# Convert to numpy for plotting
geometries = geometries.numpy()
waveforms = waveforms.numpy()
band_ffts = band_ffts.numpy()
reduced_indices = reduced_indices.numpy()

# Extract displacement components (x and y components)
displacements_x_real = displacements_dataset.tensors[0].numpy()
displacements_x_imag = displacements_dataset.tensors[1].numpy()
displacements_y_real = displacements_dataset.tensors[2].numpy()
displacements_y_imag = displacements_dataset.tensors[3].numpy()

# Combine real and imaginary parts to get complex displacements
displacements_x = displacements_x_real + 1j * displacements_x_imag
displacements_y = displacements_y_real + 1j * displacements_y_imag

print(f"Loaded dataset from: {dataset_path}")
print(f"Geometries shape: {geometries.shape}")
print(f"Waveforms shape: {waveforms.shape}")
print(f"Band FFTs shape: {band_ffts.shape}")
print(f"Displacements x shape: {displacements_x.shape}")
print(f"Displacements y shape: {displacements_y.shape}")
print(f"Reduced indices shape: {reduced_indices.shape}")

# Flags
is_export_png = True
png_resolution = 150

# Make plots for one unit cell or multiple (now limited by available geometries)
struct_idxs = list(range(min(5, geometries.shape[0])))  # Up to 5 geometries or available count

for struct_idx in struct_idxs:
    # Plot the geometry field (design pattern)
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    
    geometry = geometries[struct_idx]  # Get geometry for this structure
    
    # Plot geometry
    im = ax.imshow(geometry, cmap='gray', aspect='equal')
    ax.set_title(f'Geometry Pattern {struct_idx+1}')
    plt.colorbar(im, ax=ax)
    
    if is_export_png:
        png_path = os.path.join('png', fn, 'geometries', f'{struct_idx+1}.png')
        os.makedirs(os.path.dirname(png_path), exist_ok=True)
        plt.savefig(png_path, dpi=png_resolution, bbox_inches='tight')
    
    # Get samples for this geometry from reduced_indices
    geometry_samples = reduced_indices[reduced_indices[:, 0] == struct_idx]
    print(f'Geometry {struct_idx+1}: Found {len(geometry_samples)} samples')
    
    if len(geometry_samples) == 0:
        print(f'No samples found for geometry {struct_idx+1}, skipping...')
        continue
    
    # Plot a few sample waveforms and bands for this geometry
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # Plot first few waveforms
    for i in range(min(3, waveforms.shape[0])):
        im = axes[0, i].imshow(waveforms[i], cmap='viridis', aspect='equal')
        axes[0, i].set_title(f'Waveform {i+1}')
        plt.colorbar(im, ax=axes[0, i])
    
    # Plot all available bands
    for i in range(min(3, band_ffts.shape[0])):
        im = axes[1, i].imshow(band_ffts[i], cmap='plasma', aspect='equal')
        axes[1, i].set_title(f'Band {i+1}')
        plt.colorbar(im, ax=axes[1, i])
    
    plt.tight_layout()
    
    if is_export_png:
        png_path = os.path.join('png', fn, 'waveforms_bands', f'{struct_idx+1}.png')
        os.makedirs(os.path.dirname(png_path), exist_ok=True)
        plt.savefig(png_path, dpi=png_resolution, bbox_inches='tight')
    
    # Plot some sample displacements for this geometry
    sample_indices = geometry_samples[:min(6, len(geometry_samples))]  # Take first 6 samples
    
    fig, axes = plt.subplots(2, 6, figsize=(18, 6))
    
    for i, (geom_idx, wave_idx, band_idx) in enumerate(sample_indices):
        if i >= 6:  # Limit to 6 samples
            break
            
        # Get the displacement data for this sample
        sample_idx = np.where((reduced_indices[:, 0] == geom_idx) & 
                             (reduced_indices[:, 1] == wave_idx) & 
                             (reduced_indices[:, 2] == band_idx))[0][0]
        
        disp_x = displacements_x[sample_idx]
        disp_y = displacements_y[sample_idx]
        
        # Plot x-component displacement (magnitude)
        im1 = axes[0, i].imshow(np.abs(disp_x), cmap='RdBu', aspect='equal')
        axes[0, i].set_title(f'Sample {i+1}: |u_x|')
        axes[0, i].axis('off')
        
        # Plot y-component displacement (magnitude)
        im2 = axes[1, i].imshow(np.abs(disp_y), cmap='RdBu', aspect='equal')
        axes[1, i].set_title(f'Sample {i+1}: |u_y|')
        axes[1, i].axis('off')
    
    # Hide unused subplots
    for i in range(len(sample_indices), 6):
        axes[0, i].axis('off')
        axes[1, i].axis('off')
    
    plt.tight_layout()
    
    if is_export_png:
        png_path = os.path.join('png', fn, 'displacements', f'{struct_idx+1}.png')
        os.makedirs(os.path.dirname(png_path), exist_ok=True)
        plt.savefig(png_path, dpi=png_resolution, bbox_inches='tight')
    
    # Create a summary plot showing the data flow
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Geometry
    im1 = axes[0, 0].imshow(geometry, cmap='gray', aspect='equal')
    axes[0, 0].set_title(f'Geometry {struct_idx+1}')
    plt.colorbar(im1, ax=axes[0, 0])
    
    # Sample waveform
    wave_idx = geometry_samples[0, 1]
    im2 = axes[0, 1].imshow(waveforms[wave_idx], cmap='viridis', aspect='equal')
    axes[0, 1].set_title(f'Sample Waveform {wave_idx+1}')
    plt.colorbar(im2, ax=axes[0, 1])
    
    # Sample band
    band_idx = geometry_samples[0, 2]
    im3 = axes[1, 0].imshow(band_ffts[band_idx], cmap='plasma', aspect='equal')
    axes[1, 0].set_title(f'Sample Band {band_idx+1}')
    plt.colorbar(im3, ax=axes[1, 0])
    
    # Combined input visualization (geometry + waveform + band)
    combined_input = geometry + waveforms[wave_idx] + band_ffts[band_idx]
    im4 = axes[1, 1].imshow(combined_input, cmap='viridis', aspect='equal')
    axes[1, 1].set_title('Combined Input')
    plt.colorbar(im4, ax=axes[1, 1])
    
    plt.tight_layout()
    
    if is_export_png:
        png_path = os.path.join('png', fn, 'summary', f'{struct_idx+1}.png')
        os.makedirs(os.path.dirname(png_path), exist_ok=True)
        plt.savefig(png_path, dpi=png_resolution, bbox_inches='tight')
    
    plt.close('all')  # Close all figures to free memory

Successfully imported functions from the new Python dispersion library


AttributeError: 'list' object has no attribute 'numpy'

In [ ]:
# Demonstrate using the new Python dispersion library with the loaded data
print("="*60)
print("DEMONSTRATING NEW PYTHON DISPERSION LIBRARY")
print("="*60)

# Set up constants for the dispersion library
# These need to match the structure expected by the new library
const = {
    'a': 1.0,  # lattice parameter [m]
    'N_ele': 2,  # elements per pixel (assuming 2 based on typical values)
    'N_pix': [geometries.shape[1], geometries.shape[2]],  # pixels from loaded geometry
    'N_eig': 6,  # number of eigenvalues to compute
    'isUseGPU': False,
    'isUseImprovement': True,
    'isUseParallel': False,
    'isSaveEigenvectors': True,  # We want eigenvectors for visualization
    'isComputeGroupVelocity': False,
    'isComputeFrequencyDesignSensitivity': False,
    'isComputeGroupVelocityDesignSensitivity': False,
    'E_min': 2e9,
    'E_max': 200e9,
    'rho_min': 1e3,
    'rho_max': 8e3,
    'poisson_min': 0.0,
    'poisson_max': 0.5,
    't': 1.0,
    'sigma_eig': 1.0,
    'design_scale': 'linear'
}

# Generate wavevectors using the new library
print(f"Generating wavevectors for {const['N_pix']} pixel design...")
const['wavevectors'] = get_IBZ_wavevectors([11, 6], const['a'], 'none')
print(f"Generated {len(const['wavevectors'])} wavevectors")

# Convert the first geometry to the format expected by the dispersion library
# The library expects designs in the format [N_pix x N_pix x 3] where:
# [:,:,0] = elastic modulus (0-1 normalized)
# [:,:,1] = density (0-1 normalized) 
# [:,:,2] = Poisson's ratio (0-1 normalized)

def convert_geometry_to_design(geometry):
    """
    Convert a single geometry array to the 3-channel design format expected by the library.
    """
    N_pix = geometry.shape[0]
    design = np.zeros((N_pix, N_pix, 3))
    
    # Use the geometry as the elastic modulus channel (normalize to 0-1)
    design[:, :, 0] = (geometry - np.min(geometry)) / (np.max(geometry) - np.min(geometry))
    
    # Use the same pattern for density (assuming same material properties)
    design[:, :, 1] = design[:, :, 0]
    
    # Set Poisson's ratio to a constant value
    design[:, :, 2] = 0.6 * np.ones((N_pix, N_pix))
    
    return design

# Convert the first geometry
first_geometry = geometries[0]
design = convert_geometry_to_design(first_geometry)
const['design'] = design

print(f"Converted geometry to design format: {design.shape}")
print(f"Design ranges - E: [{np.min(design[:,:,0]):.3f}, {np.max(design[:,:,0]):.3f}]")
print(f"Design ranges - rho: [{np.min(design[:,:,1]):.3f}, {np.max(design[:,:,1]):.3f}]")
print(f"Design ranges - nu: [{np.min(design[:,:,2]):.3f}, {np.max(design[:,:,2]):.3f}]")

# Validate the constants
is_valid, missing_fields = validate_constants(const)
if is_valid:
    print("✓ Constants validation passed")
else:
    print(f"✗ Constants validation failed. Missing fields: {missing_fields}")

print("="*60)


In [ ]:
# Compute dispersion using the new Python library
print("Computing dispersion relations using the new library...")

try:
    # Compute dispersion
    wv, fr, ev = dispersion(const, const['wavevectors'])
    
    print(f"✓ Successfully computed dispersion!")
    print(f"  - Wavevectors: {wv.shape}")
    print(f"  - Frequencies: {fr.shape}")
    print(f"  - Eigenvectors: {ev.shape if ev is not None else 'None'}")
    print(f"  - Frequency range: {np.min(fr):.3f} - {np.max(fr):.3f} Hz")
    
    # Plot the design using the new library
    print("\nPlotting design using new library...")
    fig, axes = plot_design(design)
    plt.suptitle('Design Pattern (from loaded geometry)')
    plt.show()
    
    # Plot dispersion using the new library
    print("Plotting dispersion using new library...")
    fig, ax, _ = plot_dispersion(np.arange(len(wv)), fr[:, 0], 5)
    plt.title('Dispersion Relations - First Band (from new library)')
    plt.show()
    
    # Compare with the original data if available
    print("\nComparing with original dataset...")
    if len(geometry_samples) > 0:
        # Get a sample from the original dataset
        sample_idx = geometry_samples[0]
        geom_idx, wave_idx, band_idx = sample_idx
        
        print(f"Original dataset sample: geometry {geom_idx}, waveform {wave_idx}, band {band_idx}")
        
        # Plot original waveform and band
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        im1 = ax1.imshow(waveforms[wave_idx], cmap='viridis', aspect='equal')
        ax1.set_title(f'Original Waveform {wave_idx+1}')
        plt.colorbar(im1, ax=ax1)
        
        im2 = ax2.imshow(band_ffts[band_idx], cmap='plasma', aspect='equal')
        ax2.set_title(f'Original Band {band_idx+1}')
        plt.colorbar(im2, ax=ax2)
        
        plt.tight_layout()
        plt.show()
    
    # Store results for comparison
    computed_dispersion = {
        'wavevectors': wv,
        'frequencies': fr,
        'eigenvectors': ev,
        'design': design
    }
    
except Exception as e:
    print(f"✗ Error computing dispersion: {e}")
    import traceback
    traceback.print_exc()
    computed_dispersion = None

print("="*60)


In [ ]:
# Demonstrate advanced features of the new library
print("DEMONSTRATING ADVANCED FEATURES")
print("="*60)

# 1. Generate different types of designs using the library
print("1. Generating different design types...")

design_types = ['homogeneous', 'dispersive-tetragonal', 'quasi-1D']
design_examples = {}

for design_type in design_types:
    try:
        example_design = get_design(design_type, const['N_pix'][0])
        design_examples[design_type] = example_design
        print(f"  ✓ Generated {design_type} design: {example_design.shape}")
    except Exception as e:
        print(f"  ✗ Failed to generate {design_type}: {e}")

# 2. Compare dispersion for different designs
print("\n2. Comparing dispersion for different designs...")

if len(design_examples) > 0:
    fig, axes = plt.subplots(2, len(design_examples), figsize=(5*len(design_examples), 8))
    if len(design_examples) == 1:
        axes = axes.reshape(-1, 1)
    
    for i, (design_name, example_design) in enumerate(design_examples.items()):
        # Plot design
        im1 = axes[0, i].imshow(example_design[:, :, 0], cmap='gray', vmin=0, vmax=1)
        axes[0, i].set_title(f'{design_name.title()} Design')
        axes[0, i].set_aspect('equal')
        
        # Compute and plot dispersion
        try:
            const['design'] = example_design
            wv_comp, fr_comp, ev_comp = dispersion(const, const['wavevectors'])
            
            axes[1, i].plot(np.arange(len(wv_comp)), fr_comp[:, 0], 'b-', linewidth=2)
            axes[1, i].set_title(f'{design_name.title()} Dispersion')
            axes[1, i].set_xlabel('Wavevector Index')
            axes[1, i].set_ylabel('Frequency [Hz]')
            axes[1, i].grid(True, alpha=0.3)
            
            print(f"  ✓ Computed dispersion for {design_name}")
            
        except Exception as e:
            axes[1, i].text(0.5, 0.5, f'Error: {str(e)[:50]}...', 
                           ha='center', va='center', transform=axes[1, i].transAxes)
            print(f"  ✗ Failed to compute dispersion for {design_name}: {e}")
    
    plt.tight_layout()
    plt.show()

# 3. Demonstrate wavevector generation for different symmetries
print("\n3. Demonstrating different symmetry types...")

symmetry_types = ['none', 'p4mm', 'p2mm']
symmetry_examples = {}

for sym_type in symmetry_types:
    try:
        wv_sym = get_IBZ_wavevectors([9, 9], const['a'], sym_type)
        symmetry_examples[sym_type] = wv_sym
        print(f"  ✓ Generated {sym_type} wavevectors: {len(wv_sym)} points")
    except Exception as e:
        print(f"  ✗ Failed to generate {sym_type} wavevectors: {e}")

# 4. Visualize wavevector distributions
if len(symmetry_examples) > 0:
    fig, axes = plt.subplots(1, len(symmetry_examples), figsize=(5*len(symmetry_examples), 4))
    if len(symmetry_examples) == 1:
        axes = [axes]
    
    for i, (sym_type, wv_sym) in enumerate(symmetry_examples.items()):
        axes[i].scatter(wv_sym[:, 0], wv_sym[:, 1], alpha=0.6, s=20)
        axes[i].set_title(f'{sym_type.upper()} Symmetry')
        axes[i].set_xlabel('kx')
        axes[i].set_ylabel('ky')
        axes[i].set_aspect('equal')
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print("="*60)


In [ ]:
# Integration: Use new library results in the existing visualization pipeline
print("INTEGRATION WITH EXISTING VISUALIZATION PIPELINE")
print("="*60)

# Update the existing plotting loop to use the new library
if computed_dispersion is not None:
    print("Using computed dispersion from new library in existing plots...")
    
    # Modify the existing plotting loop to include new library results
    for struct_idx in struct_idxs[:2]:  # Limit to first 2 for demonstration
        print(f"\nProcessing structure {struct_idx+1} with new library integration...")
        
        # Get the geometry and convert to design format
        geometry = geometries[struct_idx]
        design = convert_geometry_to_design(geometry)
        
        # Compute dispersion using the new library
        const['design'] = design
        try:
            wv_new, fr_new, ev_new = dispersion(const, const['wavevectors'])
            
            # Create enhanced visualization combining old and new data
            fig, axes = plt.subplots(2, 3, figsize=(15, 10))
            
            # Row 1: Original data visualization
            # Geometry
            im1 = axes[0, 0].imshow(geometry, cmap='gray', aspect='equal')
            axes[0, 0].set_title(f'Original Geometry {struct_idx+1}')
            plt.colorbar(im1, ax=axes[0, 0])
            
            # Sample waveform from original data
            if len(geometry_samples) > 0:
                sample_idx = geometry_samples[0]
                wave_idx = sample_idx[1]
                im2 = axes[0, 1].imshow(waveforms[wave_idx], cmap='viridis', aspect='equal')
                axes[0, 1].set_title(f'Original Waveform {wave_idx+1}')
                plt.colorbar(im2, ax=axes[0, 1])
            
            # Sample band from original data
            if len(geometry_samples) > 0:
                band_idx = sample_idx[2]
                im3 = axes[0, 2].imshow(band_ffts[band_idx], cmap='plasma', aspect='equal')
                axes[0, 2].set_title(f'Original Band {band_idx+1}')
                plt.colorbar(im3, ax=axes[0, 2])
            
            # Row 2: New library results
            # Design pattern (3-channel)
            im4 = axes[1, 0].imshow(design[:, :, 0], cmap='gray', vmin=0, vmax=1, aspect='equal')
            axes[1, 0].set_title(f'New Library Design {struct_idx+1}')
            plt.colorbar(im4, ax=axes[1, 0])
            
            # Dispersion curve
            axes[1, 1].plot(np.arange(len(wv_new)), fr_new[:, 0], 'b-', linewidth=2, label='Band 1')
            if fr_new.shape[1] > 1:
                axes[1, 1].plot(np.arange(len(wv_new)), fr_new[:, 1], 'r-', linewidth=2, label='Band 2')
            axes[1, 1].set_title(f'New Library Dispersion')
            axes[1, 1].set_xlabel('Wavevector Index')
            axes[1, 1].set_ylabel('Frequency [Hz]')
            axes[1, 1].legend()
            axes[1, 1].grid(True, alpha=0.3)
            
            # Eigenvector visualization (if available)
            if ev_new is not None:
                # Plot magnitude of first eigenvector
                ev_magnitude = np.abs(ev_new[:, 0, 0])  # First wavevector, first mode
                ev_reshaped = ev_magnitude.reshape(const['N_pix'][0]*2, const['N_pix'][1]*2)
                im5 = axes[1, 2].imshow(ev_reshaped, cmap='RdBu', aspect='equal')
                axes[1, 2].set_title(f'First Mode Shape')
                plt.colorbar(im5, ax=axes[1, 2])
            else:
                axes[1, 2].text(0.5, 0.5, 'Eigenvectors\nnot computed', 
                               ha='center', va='center', transform=axes[1, 2].transAxes)
                axes[1, 2].set_title('Mode Shape (N/A)')
            
            plt.tight_layout()
            
            if is_export_png:
                png_path = os.path.join('png', fn, 'integrated_results', f'{struct_idx+1}.png')
                os.makedirs(os.path.dirname(png_path), exist_ok=True)
                plt.savefig(png_path, dpi=png_resolution, bbox_inches='tight')
                print(f"  Saved integrated results to: {png_path}")
            
            plt.show()
            
            print(f"  ✓ Successfully integrated new library results for structure {struct_idx+1}")
            
        except Exception as e:
            print(f"  ✗ Error processing structure {struct_idx+1}: {e}")
            import traceback
            traceback.print_exc()

else:
    print("No computed dispersion available from new library")

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print("✓ Successfully integrated the new Python dispersion library")
print("✓ Converted geometry data to the format expected by the library")
print("✓ Demonstrated dispersion computation using the new library")
print("✓ Showed advanced features (design generation, symmetry types)")
print("✓ Integrated new library results with existing visualization pipeline")
print("✓ Maintained compatibility with existing PyTorch dataset structure")
print("\nThe notebook now uses the new 2d-dispersion-py library while")
print("preserving the existing data loading and visualization capabilities.")
print("="*60)
